In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Locate the data
data = '../Nasdaq Std.csv'
# Read data with pandas
dataset = pd.read_csv(data)
# Extract the values of the desired column
full_set = dataset['NAS_Returns_Rolling_Std'].values
dataset

,Unnamed: 0,Date,NASDAQ_100,NAS_Returns_Rolling_Std
0,0,1979-12-28,0.399392,NaN
1,1,1979-12-31,0.205527,NaN
2,2,1980-01-02,-1.965066,NaN
3,3,1980-01-03,-1.484779,NaN
4,4,1980-01-04,1.404400,NaN
...,...,...,...,...
10366,10366,2021-02-08,0.947871,1.014844
10367,10367,2021-02-09,0.143416,1.014787
10368,10368,2021-02-10,-0.251076,1.013661
10369,10369,2021-02-11,0.381028,1.012610


In [3]:
# We split our data on the 7200 mark as it represents 70% of it
train_data = full_set[5061:9061]
test_data = full_set[9061:]
# dataset.iloc[5061]

In [4]:
# Data Normalization
from sklearn.preprocessing import MinMaxScaler
# Data must be reshaped in order to use fit_transform
training_set = train_data.reshape(-1,1)
test_set = test_data.reshape(-1,1)

sc = MinMaxScaler(feature_range=(0,1))

X_train_scaled = sc.fit_transform(training_set)

In [5]:
# Create lists for x and y train
X_train = []

y_train = []

# We will run our model on 100 timesteps
for i in range(20,4000):
    
    X_train.append(X_train_scaled[i- 20:i,0])

    y_train.append(X_train_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [6]:
# Import the necessary models from keras within tensor flow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [7]:
# Create a callback function with keras, which will monitor thr rmse of each epoch 
# And stop our model on the epoch that offers the best balance so we don't overfit our model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [8]:
# Create the layers of our model droping out and replenishing 20% of our data after each layer
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
# Include a printout of the model summary 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [9]:
model.compile(optimizer='adam',loss= 'mean_squared_error',
             metrics=[tf.keras.metrics.RootMeanSquaredError()])
# While we git our model we assign it to a variable 
history = model.fit(X_train,y_train,epochs=20,batch_size=100,callbacks=[callback], verbose = 2)

Epoch 1/20
40/40 - 12s - loss: 0.0146 - root_mean_squared_error: 0.1206
Epoch 2/20
40/40 - 2s - loss: 0.0038 - root_mean_squared_error: 0.0614
Epoch 3/20
40/40 - 2s - loss: 0.0029 - root_mean_squared_error: 0.0541
Epoch 4/20
40/40 - 2s - loss: 0.0027 - root_mean_squared_error: 0.0519
Epoch 5/20
40/40 - 2s - loss: 0.0022 - root_mean_squared_error: 0.0469
Epoch 6/20
40/40 - 2s - loss: 0.0020 - root_mean_squared_error: 0.0447
Epoch 7/20
40/40 - 2s - loss: 0.0018 - root_mean_squared_error: 0.0419
Epoch 8/20
40/40 - 2s - loss: 0.0017 - root_mean_squared_error: 0.0415
Epoch 9/20
40/40 - 2s - loss: 0.0017 - root_mean_squared_error: 0.0411
Epoch 10/20
40/40 - 2s - loss: 0.0014 - root_mean_squared_error: 0.0375
Epoch 11/20
40/40 - 2s - loss: 0.0014 - root_mean_squared_error: 0.0377
Epoch 12/20
40/40 - 2s - loss: 0.0012 - root_mean_squared_error: 0.0353
Epoch 13/20
40/40 - 2s - loss: 0.0013 - root_mean_squared_error: 0.0355
Epoch 14/20
40/40 - 2s - loss: 0.0013 - root_mean_squared_error: 0.0366


In [10]:
# Print out the number of epochs used for this model
print(len(history.history['loss']))

20


In [11]:
# Making Predictions to the test set
dataset_total = np.concatenate((train_data, test_data), axis = 0)
# Prepare our data to the test
inputs = dataset_total[len(dataset_total) - len(test_data) - 100:]
# Reshape our data to fit the upcoming for loop
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

# Create lists to which we'll append x and y values for future test
X_test = []
y_test = []

# Create a loop that will run through groups of 200 timesteps for the test sample
for i in range(20, 1410):
    X_test.append(inputs[i-20:i,0])
    y_test.append(inputs[i, 0])

In [12]:
# Turn our lists into a numpy numbers array
X_test = np.array(X_test)
y_test = np.array(y_test)

# Reshape our array into 3 dimensions so it can fit the LSTM Model
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_test = y_test.reshape(-1, 1)

In [13]:
 model_loss, rmse = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Root Mean Square Error:{rmse}")

44/44 - 3s - loss: 0.0011 - root_mean_squared_error: 0.0330
Normal Neural Network - Loss: 0.0010867305099964142, Root Mean Square Error:0.03296559676527977


In [17]:
predicted_volatility = model.predict(X_test)
predicted_volatility = sc.inverse_transform(predicted_volatility)

In [18]:
predicted_volatility

array([[0.85745966],
       [0.8584431 ],
       [0.85865664],
       ...,
       [1.0431296 ],
       [1.067163  ],
       [1.0828918 ]], dtype=float32)

In [19]:
y_test = sc.inverse_transform(y_test)


In [20]:
# predicted_volatility = predicted_volatility * 100

In [21]:

import plotly.express as px

In [22]:
output = pd.DataFrame({
    'predicted_volatility' : [x[0] for x in predicted_volatility],
    'actual_volatility' : [x[0] for x in y_test]
})
output

,predicted_volatility,actual_volatility
0,0.857460,3.983323
1,0.858443,4.036881
2,0.858657,4.115889
3,0.859162,4.116564
4,0.860817,4.107798
...,...,...
1385,0.975271,4.816555
1386,1.011713,4.816319
1387,1.043130,4.811646
1388,1.067163,4.807285


In [25]:
px.line(output, y=["predicted_volatility", "actual_volatility"])

In [24]:
output.to_csv('../../Model Results/Nasdaq-Volatility-Predicted.csv')
